In [1]:
from fiscalsim_us import IndividualSim
import pandas as pd
import plotly.express as px

In [12]:
def make_tax(adults, children):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse", age=25)
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members, premium_tax_credit=0, mn_gambling_loss_deduction=100_000)
    sim.add_household(name="household", members=members, state_code="MN")
    sim.vary("employment_income", max=600_000, step=10_000)
    employment_income = sim.calc("employment_income")[0]
    state_income_tax = sim.calc("state_income_tax")[0].round()
    mtr = sim.deriv("state_income_tax", "employment_income", wrt_target="head")
    amt = sim.calc("mn_amt")[0]
    basic_tax = sim.calc("mn_basic_tax")[0]
    return pd.DataFrame(
        dict(
            employment_income=employment_income,
            state_income_tax=state_income_tax,
            mtr=mtr,
            amt=amt, 
            basic_tax=basic_tax,
            adults=adults,
            children=str(children),
        )
    )


# Make a table of state taxes for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 8):
        l.append(make_tax(adults, children))

In [13]:
df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    state_income_tax="Minnesota income tax",
    mtr="Marginal tax rate",
    adults="Adults",
    children="Children",
    amt="Alternative income tax",
    basic_tax="Minnesota income tax without AMT"
)
LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}

fig = px.line(
    df,
    "employment_income",
    "amt",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Minnesota Alternative Minimum Tax",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

In [4]:
fig = px.line(
    df,
    "employment_income",
    "state_income_tax",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Minnesota income tax",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

In [5]:
fig = px.line(
    df,
    "employment_income",
    "basic_tax",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Minnesota income tax without amt",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

## Vary "unallowed" deductions

In [6]:
def make_tax_2(adults, children):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25, employment_income=200_000)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse", age=25)
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members, premium_tax_credit=0)
    sim.add_household(name="household", members=members, state_code="MN")
    sim.vary("mn_other_itemized_deductions", max=200_000, step=10_000)
    deducts = sim.calc("mn_other_itemized_deductions")[0]
    state_income_tax = sim.calc("state_income_tax")[0].round()
    mtr = sim.deriv("state_income_tax", "employment_income", wrt_target="head")
    amt = sim.calc("mn_amt")[0]
    basic_tax = sim.calc("mn_basic_tax")[0]
    return pd.DataFrame(
        dict(
            deducts=deducts,
            state_income_tax=state_income_tax,
            mtr=mtr,
            amt=amt, 
            basic_tax=basic_tax,
            adults=adults,
            children=str(children),
        )
    )


# Make a table of state taxes for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 8):
        l.append(make_tax_2(adults, children))

In [10]:
df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    state_income_tax="Minnesota income tax",
    mtr="Marginal tax rate",
    adults="Adults",
    children="Children",
    amt="Alternative minimum tax",
    basic_tax="Minnesota income tax without AMT",
    deduts="Minnesota itemizeded deductions"
)
LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}

fig = px.line(
    df,
    "deducts",
    "state_income_tax",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Minnesota income tax (with AMT)",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

In [8]:
fig = px.line(
    df,
    "deducts",
    "amt",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Minnesota alternative minimum tax",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

In [9]:
fig = px.line(
    df,
    "deducts",
    "basic_tax",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Minnesota income tax without AMT",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()